In [1]:
import yaml
import numpy as np

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import directed_hausdorff

from data.dataset import NUFTDataset
from train.nuft_trainer import Trainer
from model.nn import build_model
from torchinfo import summary

/data/anaconda3/envs/gdl/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
with open('cfg/nuft.yaml', 'r') as f:
    cfg = yaml.safe_load(f)  
    
import pandas as pd
osm_df = pd.read_pickle(cfg['osm'])
melb_df = pd.read_pickle(cfg['melb'])

osm_df_o = osm_df[osm_df.trans == 'o'].reset_index(drop=True)    
osm_df_r = osm_df[osm_df.trans == 'r'].reset_index(drop=True)  

osm_set_o = NUFTDataset(osm_df_o).to_data_list()
osm_set_r = NUFTDataset(osm_df_r).to_data_list()
melb_set = NUFTDataset(melb_df).to_data_list()

datasets = {'osm_set_o': osm_set_o, 
            'osm_set_r': osm_set_r,
            'melb_set': melb_set
            }

backbone = ['nuft']

ckpt = 'epoch100'

  0%|          | 0/5010 [00:00<?, ?it/s]/data/anaconda3/envs/gdl/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 37139/37139 [00:44<00:00, 830.14it/s]


In [3]:

cfg['nn'] = 'nuft'
cfg['path'] = f'save/{cfg["nn"]}'

model = build_model(cfg=cfg)
trainer = Trainer(cfg=cfg) 
model = trainer.load_ckpt(model, 'epoch100')
model.eval()
print(summary(model))

metrics = {}
for key, dataset in datasets.items():
    zs = []
    for data in dataset:
        data.to('cuda')
        z = model.encoder(data.x, data.batch)
        zs.append(z.mean(0).detach().cpu().numpy())

    zs = np.stack(zs, axis=0)
    zs = zs[~np.isnan(zs).any(axis=1)]
    
    # NN neighbors
    nbrs = NearestNeighbors(n_neighbors=7, algorithm='auto').fit(zs)
    dist, idx = nbrs.kneighbors(zs)
    # Hausdorff dist    
    query_id = idx[:, 0]
    target_id = idx[:, 1:]
    haus_dist = []
    for id, tids in enumerate(target_id):
        qid = query_id[id]
        tem_dist = []
        for tid in tids: 
            dist = directed_hausdorff(dataset[qid].x.cpu().numpy(), 
                                      dataset[tid].x.cpu().numpy())
            tem_dist.append(dist[0])
        haus_dist.append(tem_dist)
    metrics[key] =  np.stack(haus_dist, axis=0).mean(axis=0).tolist()
    
nuft = {'nuft': metrics}
nuft

Layer (type:depth-idx)                   Param #
NUFTAE                                   --
├─MLP: 1-1                               --
│    └─ReLU: 2-1                         --
│    └─ModuleList: 2-2                   --
│    │    └─Linear: 3-1                  73,984
│    │    └─Linear: 3-2                  32,896
│    │    └─Linear: 3-3                  8,256
│    └─ModuleList: 2-3                   --
│    │    └─BatchNorm: 3-4               512
│    │    └─BatchNorm: 3-5               256
├─MLP: 1-2                               --
│    └─ReLU: 2-4                         --
│    └─ModuleList: 2-5                   --
│    │    └─Linear: 3-6                  8,320
│    │    └─Linear: 3-7                  33,024
│    │    └─Linear: 3-8                  74,016
│    └─ModuleList: 2-6                   --
│    │    └─BatchNorm: 3-9               256
│    │    └─BatchNorm: 3-10              512
Total params: 232,032
Trainable params: 232,032
Non-trainable params: 0


{'nuft': {'osm_set_o': [0.18049155195738995,
   0.19494331552352276,
   0.20289117309174384,
   0.20918209830345272,
   0.21546777533303385,
   0.21916361310216248],
  'osm_set_r': [0.23258541162092633,
   0.2466422720493908,
   0.2546385274028193,
   0.26030236531666645,
   0.2634298720698794,
   0.26738596147334504],
  'melb_set': [0.12728232158695982,
   0.13986859449958855,
   0.14648889308990592,
   0.1509600343076432,
   0.15433366955630676,
   0.15696956418383526]}}